<span type="title">类型信息</span> | <span type="update">2018-07-28</span> | <span type="version">1.0</span>
<span type="intro"><p class="card-text">本章主要介绍Java运行时类型信息RTTI。首先从RTTI的机制和必要性谈起，接着介绍了其工作的原理：利用Class类对象。对于Class类对象，提到了Java类加载、链接、初始化的步骤。介绍了使用forName和类字面常量获取类对象引用的方法，特别提到了泛化类对象的引用。</p><p class="card-text">在第二部分，着重讲解了 instanceof、isInstance(obj), isAssignableFrom(Class) 进行类型检查的方法，并且给出一个具体例子来说明这些方法和泛型类对象是如何被使用的。之后讲解了RTTI的类字面量对于继承的工厂类带来的改进。</p><p class="card-text">在第三部分介绍了RTTI反射的基本知识，同时举了三个例子：遍历类文件获得其方法、动态加载和构造类、简化反射设计模式。我们使用了一个用来获取类方法的小工具 GetMethod.java 介绍了 reflect包中 Method Constructor 类的使用方法。之后的例子提到了如何动态加载和初始化字节码的方法，最后讲解了基于反射的动态代理对于代理模式带来的改进。</p><p class="card-text">在最后一部分，介绍了空对象、以及我们无法避免程序员使用RTTI破坏接口来访问Java的任意位置的任意方法的原因。</p></span>

# RTTI 运行时类型信息

RTTI 允许我们在实际运行的时候才发现和知晓类型的相关信息。Java 用两种 RTTI，传统的 RTTI 假定在编译的时候，我们已经知道了所有的类型（虽然不知道类型的所有信息）。另一种是 `反射`，反射允许我们在运行时（编译之后）从文件、流或者网络获取 `.class` 二进制文件，发现并且动态使用这些类型信息（在程序运行的时候，它依旧可以加载组件，并且让组件和它的代码共同工作）。

**RTTI(Runtime Type Info)的机制**： RTTI 提供了一种对于接口/基类调用，而不需知道类型的面向对象编程方法。因此，编译器在进行编译的时候并不知道其方法调用实现的具体类型，这样提高了程序的易扩展性和逻辑清晰度。RTTI 的基本用途在于动态绑定时的类型识别，从而访问合适的方法调用。需要注意，这里的 RTTI 运行时类型识别并非意味着其不知道有这个实现了接口的导出类，而是说，它不知道这个对于基类接口的调用到底是指向哪一个导出类实现的。

典型的是，当你创建一个类 `Cat`，调用 `Cat c = new Cat(); c.toString()` 时，这个本来为 `Object.class` 而写的方法会因为 RTTI 而被识别出具体的类型 `Cat`，而在这个我们构造的类中寻找重载，如果找不到，就向上转型，一直转型到 Object 对象，打印出内存的地址。如果说接口是多态的表现，那么 RTTI 就是向上转型和多态机制的具体执行步骤：通过运行时类型识别调用导出类的方法。

**为什么我们需要RTTI？** 对于多态的实现，RTTI 提供了动态绑定的机制。那么，对于程序编写人员，为什么需要RTTI呢？按照理想情况，我们只需要和接口打交道，所有的导出类都应该只实现接口的方法，通过接口调用，而Java会在运行时识别类型，进行动态绑定和调用。这是OO的核心。

但是，总有一些时候，我们需要指定某个类来做某些事，在这样的情况下，使用接口并不方便，或者说，有时候我们无法修改接口或者基类。此外，对于一些问题，使用RTTI解决起来更加方便，比如带有继承的工厂等。在下面会介绍到。


# Class 对象

`Class` 是作为 RTTI 的基本单元。如果想要理解 RTTI 在 Java 中的工作原理（也就是 Java 动态绑定的机制），那么就必须了解这些类型是如何表示的：类对象。类对象就是用来创建常规类的所有常规对象的，Java 使用类对象来执行 RTTI。每个类文件 `.class` 在运行的时候都会相伴一个类对象 `Class<T>`。类对象是一个 `Class` 类的对象，这个对象中保存了关于类文件中这个类的具体信息，之后对这个类进行初始化、构造器构造等创建类的实例的方式都要通过这个 Class对象。（为了方便叙述，类对象、Class对象在下文指的是Class类的对象，而类的对象则称之为类示例或者类的对象）。

JVM 包含一系列的 类加载器 用来加载不同的类，其中原生类加载器用来加载 Java API 的类。

类并非是在 Java 程序运行时就自动全部加载(不同于C++），当程序创建第一个对于类的静态成员的引用时，才会加载这个类。对于静态方法、构造器（本质也是静态的）、静态字段，都会被当作是对类静态成员的引用。类加载器会检查`.class`文件，如果没有被破坏，那么创建一个Class对象对其进行加载和初始化。当类被加载到内存中，才可以创建这个类的所有对象。

## forName 获取引用

下面的例子介绍了 Class 类中常用的方法： `forName(String)` 获取一个类的类对象 `Class x`。这个类对象拥有这个类的必要信息，比如：

- getName() 类名
- getSimpleName() 基本类名
- getMethods() 返回一个 Method[] 类型的关于这个类的方法数组
- getClassLoader() 返回加载这个类的具体类加载器
- getSuperclass() 返回超类
- getCanonicalName() 不包含包的类名
- getPackageName() 包的名字

需要注意，当我们使用 `Class x = Class.forName("Candy")` 获取一个类对象的引用的时候，并没有构造这个类的实例，你可以看到静态方法被执行了，实际上，当获取类对象的引用的时候，已经进行了加载、链接和初始化这三个步骤的动作了。我们可以通过 `getDeclaredConstructor().newInstance()` 用默认（没有参数的构造器）构造一个实例，而不用通过 new 关键字。


```java
import static com.mazhangjing.Print.*；
class Candy {
    static final double PI = 3.141592;
    static double PPI = 3.14;
    static {print("Candy loading...");}
    Candy() {print("init Candy...");}
}
class Sendy {
    static {print("Sendy loading...");}
    Sendy() {print("Init Sendy...");}
    void getName() {print("My NAME IS SENDY");}
}
public class ClassDemo {
    public static void main(String[] args) throws Exception{
        //---------------展示了使用forName()获取类对象引用的方式--------------
        new Candy(); //造成静态域执行、构造器构造的副作用
        //forName 静态方法用来根据类名查找一个类，然后取得对Class对象（而不是Sendy对象）的引用
        Class x = Class.forName("Sendy"); //只造成静态域执行的副作用，只初始化并未构造
        
        //下面是Class对象从这个加载到内存的类中获得的信息：
        printf("Name: %s, \n不包含包类名: %s, \nMethods: %s, \nclassLoader: %s, \nsuperclass: %s" +
                        "\n全限定类名: %s\npackageName: %s\n",
                x.getName(),x.getSimpleName(),x.getMethods(),
                x.getClassLoader(),x.getSuperclass(),x.getCanonicalName(),
                x.getPackageName());
                
        /**
        上述代码执行结果：
        Sendy loading...
        Name: Sendy, 
        不包含包类名: Sendy, 
        Methods: [Ljava.lang.reflect.Method;@3796751b, 
        classLoader: jdk.internal.loader.ClassLoadersAppClassLoader@4629104a, 
        superclass: class java.lang.Object
        全限定类名: Sendy
        packageName: */


        //---------------------展示了使用类对象构造类实例的方式--------------------------
        Object s = x.getDeclaredConstructor().newInstance(); //注意，默认的返回类型是Object，需要转型
        Sendy ss = (Sendy)s;
        ss.getName();//Init Sendy... My NAME IS SENDY
        }
}
```
    
## 类字面常量获取引用

其实，除了使用 `forName()` 这种方式获取类对象的引用外，还可以使用类字面常量。类字面常量的写法为：`SomeClassName.class` 。通过这种语法，可以为接口、数组、基本类型、普通类创建类对象引用。对于包装器，还可以使用 `Integer.TYPE` 这种写法，不过没有必要，使用 `Boolean.class` 一致性更好。

类的使用步骤具体有三：

- 加载：加载器从文件中加载 `.class` 文件，找到字节码，然后创建一个 Class对象（加载和创建类对象）
- 链接：验证字节码，同时为静态域分配空间，必要的话，解析到其他类的引用。（分配内存空间）
- 初始化：如果类有超类，那么对其进行初始化。此外，执行静态初始化块。（执行构造器构造前的动作）

区别于使用 `getName()` 方法，使用类字面常量的方式更为惰性。具体而言，其不会执行步骤三：初始化，因此我们看不到 `Candy loading...` 这句放在静态域中的打印语句。但是，当调用这个类字面常量创建的类对象的引用后，还是会进行步骤三的。

此外，这里需要区分一二步和第三步，前两步是在编译前完成的，第三步是在编译后完成的。对于数据而言，static final是一个编译期常量，而 static 则不是。

```java
public class ClassDemo2 {
    public static void main(String[] args) throws Exception{
        //---------------------展示类字面常量的使用-------------------------------------
        Class c = Candy.class;
       
        Class.forName("Sendy"); //使用forName会完全进行类准备，而是用类字面常量则是惰性的
        //不进行初始化，仅分配内存空间

        //static 和 static final 的表现不同，前者不是一个编译期常量，因此需要等到初始化类之后才能执行
        print(Candy.PI); //static final 的值在链接阶段已经完成，因此可以读取
        print(Candy.PPI); //为了读取这个Static数据，我们被迫进行了初始化，执行了静态初始化块内容
        

        //-------------------------展示限定类型的类对象--------------------------------
        //使用泛型语法限定指向Class引用的类型
        Class<Integer> ic = int.class; //发生了转型
        Class<Integer> ic2 = Integer.class; //等同于前者，返回的Class引用都是Integer

        //Class<Number> ic3 = int.class;
        //虽然int是number的子类，但是int class对象不是number class对象子类
        Class<?> ic3 = int.class;
        Class ic4 = int.class;//这两种等价，但是前者更好，其说明了你想要使用某种Class而不是其基类
        Class<? extends Number> ic5 = int.class; //这种方法可以保证class的引用为Number的子类

                                                            
        //------------------------使用类对象.cast()对类实例进行转型---------------------
        float a = 10;
        Number n = ic5.cast(a);//cast允许将一个对象转型为引用的Class所限定的类型
        Number b = (Number)a; // 这样写更简单，但是前者允许转型到储存在某个变量中的Class引用
    }
}
```

## 泛化类对象的引用

**限制类对象的类型**： 我们可以使用泛型来限制类对象的类型，而不是使用 Class，这样使用并不方便。使用 `Class<Integer>` 会限定这个Class对象是一个指向Integer的Class。需要注意，Number导出了Integer，但是不能 `Class<Number> = int.class` 这样写。因为`Class<Integer>`并不是`Class<Number>`的一个子类，`Integer`才是`Number`的一个子类。因此，想要限定类对象类型为某个继承树的时候，需要这样写：`Class<? extends Number>`。此外，对于单纯的Class，也推荐使用 `Class<?>` 来表示类型，这样意味着我们要的是一个具体的Class对象，而不是泛化的对象。

**使用cast()转型一个类实例**：直接转换类型更简单，这里提供了一种知道类对象引用情况下的转型方法。不常用。

# 类型检查

RTTI 不仅可以通过 Class 类对象或者直接运算符进行转型，还可以通过 `instanceof` 关键字进行判断，下面展示了使用这种方法的例子。

补充一句：对于 Class，使用 `isInstance(obj), isAssignableFrom(Class)` 可以进行类型的判断。此外，使用 `equals() ==` 运算符也可以进行判断。不过，区别于前三者可以判断继承树，运算符和 equals 只能够判断是否类型完全相等，不能判断是否有继承关系。

```java
import typeinfo.pets.*; import java.lang.reflect.Array; import java.util.*;
import static com.mazhangjing.Print.print;

class A {}
class B extends A {}
class C extends B {}

public class CheckCast {
    public static void main(String[] args) throws Exception {
        B b = new B(); A a = new A();
        print(b instanceof B); //b是B的实例 ture
        print(b instanceof A); //b是A的实例，因为B是A的继承， ture
        print(b instanceof C); //b不是C的实例 false
                                                             
        //使用 instanceof 可以安全的通过Java转型检查后，如果不出错就进行转型
        if (b instanceof A){ print((A)b); }
                                    
        //--------------------展示了使用Class的 isInstance 判断类型关系--------
        print(b.getClass().isInstance(b)); //b is instance of B, true
        //b.getClass() 得到的是一个Class<B>类的引用，这里判断的是B而不是Class<B>!
                                                             
        Class<? extends B> x = b.getClass();
        //print(b instanceof x); 因此不能这样写，x是一个类实例引用，当使用.isInstance(b)的时候，
        //Java去寻找是否 b 是我们这个Class类的具体内部类——<B>的一个实例

                                                             
        //--------------------展示了使用Class的 isAssignableFrom 判断类型关系--------
        //Class<B> isAssignableFrom Class<A> 可以理解为，B是否可以接受A的转型
        print(Class.forName("B").isAssignableFrom(a.getClass())); //B可接受A的转型吗？否，A是基类 false
        print(Class.forName("A").isAssignableFrom(b.getClass())); //A可接受B的转型。是。 ture
        
        //-------------------展示了使用equals和==判断的方法---------------
        b.getClass().equals(A.class); //false
        print(b.getClass() == B.class); //ture
    }
}
```

# RTTI 用法一览

下面一个例子包含了多个 PetCreator 的实现，这个类用来返回一个 Pet 及其继承类 宠物的随机类对象列表。而 Counter 及其不同版本则用作这个列表中宠物的分类计数。下列代码使用不同的RTTI方式，展示了完成相同工作——创建列表并计数的方法。其使用到了以上讲解的所有获取类对象的手段。也展示了限制类型（使用泛型的`Class<? extends T>`）对于制造通用工具的巨大帮助。

注意，保存Pet及其继承类的类对象的Map键值对使用 `Map.Entry<Class<? extends Pet>,Integer>` 表示。

注意，对于构造器，如果要在其前完成一些工作，放在静态域中执行代码。

## forName示例

下面展示了 `abstract class PetCreator` `class ForNameCreator extends PetCreator`两个类。

其中抽象类中包含了一个Pet类及其继承类的宠物列表（这个列表并非保存宠物对象，而是保存着宠物的类对象），这个列表使用 `List<Class<? extends Pet>> types()` 方法获取，且必须子类实现。此外，这个抽象类包含了使用这个列表创建随机宠物的三个方法 `public Pet randomPet()` `public Pet[] createArray(int size)` `public ArrayList<Pet> arrayList(int size)`。这些方法并非抽像，可以获取不同数量的随机宠物列表/数组。

继承类 `ForNameCreator` 的主要作用就是实现这样一个包含了所有宠物类对象列表的的 `types` 方法。我们看到，为了生成这样一个包含所有宠物类对象的列表，我们在静态域中放置了一些代码，使用类名字符串和 `forName` 方法来产生这些类对象： `types.add((Class<? extends Pet>)Class.forName(i));`。

```java
//抽象基类
abstract class PetCreator {
    private Random rand = new Random();
    public abstract List<Class<? extends Pet>> types();
    public Pet randomPet() {
        int n = rand.nextInt(types().size());
        try {
            return types().get(n).getDeclaredConstructor().newInstance();
        } catch (Exception e) {
            throw new RuntimeException(e);
        }
    }
    public Pet[] createArray(int size) {
        Pet[] result = new Pet[size];
        for (int i = 0; i < size; i++){
            result[i] = randomPet();
        }
        return result;
    }
    public ArrayList<Pet> arrayList(int size) {
        ArrayList<Pet> a = new ArrayList<>();
        Collections.addAll(a,createArray(size));
        return a;
    }
}

//一个基本的Creator类
class ForNameCreator extends PetCreator {
    private static List<Class<? extends Pet>> types = new ArrayList<>();
    private static String[] typeNames = {
            "typeinfo.pets.Mutt",
            "typeinfo.pets.Pug",
            "typeinfo.pets.Manx",
            "typeinfo.pets.Rat",
            "typeinfo.pets.Mouse",
            "typeinfo.pets.Hamster"
    };
    @SuppressWarnings("unchecked")
    private static void loader() {
        try {
            for (String i : typeNames){
                types.add((Class<? extends Pet>)Class.forName(i));
            }
        } catch (Exception e) {
            throw new RuntimeException(e);
        }
    }
    static {loader();}//初始化时即加载loader方法，然后在构造这个对象之前，已经有了好的types，
    // 就可以直接返回了。之所以这样做的原因是，
    public List<Class<? extends Pet>> types() {
        return types;
    }
}
```
这里面需要注意的点是，我们为了静态调用 ForNameCreator，因此使用了一个静态域执行一个静态方法，而这个静态方法被用来更改一个静态的变量 types。

## 类字面量示例

其实，我们还可以使用类字面量进一步简化这个宠物构造器：

```java
//一个稍微高级一点的Creator类，展示了使用类字面常量简化代码
class LiteralPetCreator extends PetCreator {
    @SuppressWarnings("unchecked")
    //不同于typeinfo.pets.xxx, forName()需要对这些类进行初始化，因此
    //必须放置在 try 块中，而是用类字面量只到了链接级别，所以在编译期已经处理完毕，不需要捕获异常
    public static final List<Class< ? extends Pet>> allTypes =
            Collections.unmodifiableList(Arrays.asList(
                    Pet.class,Dog.class,Cat.class,Rodent.class,Mutt.class,
                    Pug.class,Manx.class,Cymric.class,Rat.class,Mouse.class,
                    Hamster.class));
    private static final List<Class<? extends Pet>> types =
            allTypes.subList(allTypes.indexOf(Mutt.class),allTypes.size());

    public List<Class<? extends Pet>> types() {
        return types;
    }
    public static void main(String[] args){
        System.out.println(types);
    }
}
```

如上所示，将所有的类字面量添加到一个List中即可完成上述创建类对象的复杂工作。

## instanceof 示例

我们来看看，这个Creator将会做什么：计数。下面这个PetCount类实现了对于宠物的分类计数。其中内部类继承自HashMap，用来用Map保存不同种类宠物的信息。我们看到它是如何识别宠物属于哪个类的：使用 `instanceof` 关键字，显然这并不方便。

```java
//一个使用静态instanceof计数的例子
class PetCount {
    static class PetCounter extends HashMap<String,Integer> {
        public void count(String type) {
            Integer quantity = get(type);
            if (quantity == null){
                put(type,1);
            } else {
                put(type,quantity + 1);
            }
        }
    }
    public static void countPets(PetCreator creator){
        PetCounter counter = new PetCounter();
        for (Pet pet: creator.createArray(20)){
            //print(pet.getClass().getSimpleName() + " ");
            if (pet instanceof Pet){
                counter.count("Pet");
            } if (pet instanceof Dog) {
                counter.count("Dog");
            } if (pet instanceof Mutt){
                counter.count("Mutt");
            } if (pet instanceof Cat) {
                counter.count("Cat");
            } if (pet instanceof Manx) {
                counter.count("Manx");
            } if (pet instanceof Rat) {
                counter.count("Rat");
            } if (pet instanceof Mouse) {
                counter.count("Mouse");
            }
        }
        print(counter);
    }
    public static void main(String[] args){
        //接受一个Creator，调用接口的createArray方法创建一个序列。而这个方法则需求types构造的列表
        countPets(new ForNameCreator());
        countPets(new LiteralPetCreator());
    }
}
```

## isInstance 示例

下面是一个改进，使用Class的 `isInstance(Object o)` 来判断类型信息。

```java
class PetCount2 {
    static class PetCounter extends LinkedHashMap<Class<? extends Pet>,Integer> {
        public PetCounter() {
            for (Class<? extends Pet> t : LiteralPetCreator.allTypes){
                this.put(t,0);} //必须先添加全部类型到Map中
        }
        public void count(Pet pet) {
            for (Map.Entry<Class<? extends Pet>,Integer> pair : entrySet()){
                if (pair.getKey().isInstance(pet)) 
                //判断类型的方法，比instanceof好用多了
                         put(pair.getKey(),pair.getValue()+1);
            }
        }
        public String toString() {
            StringBuilder sb = new StringBuilder("{");
            for (Map.Entry<Class<? extends Pet>,Integer> pair : entrySet()){
                sb.append(pair.getKey().getSimpleName());
                sb.append("=");//注意不要使用+运算符
                sb.append(pair.getValue());
                sb.append(", ");
            }
            sb.delete(sb.length()-2,sb.length());
            sb.append("}");
            return sb.toString();
        }
    }
    public static void main(String[] args){
        PetCounter petCount = new PetCounter();
        for (Pet pet : Pets.createArray(20)){
            print(pet.getClass().getSimpleName() + "");
            petCount.count(pet);
        }
        print(); print(petCount);
    }
}
```

## isAssignableFrom 和 Class(T) 示例

那么，我们能做的就这么多吗？我们看到，这个计数器和Pet耦合太深，即便是用isInstance，这个Map也必须要有所有的类型才可以。为了做到这一点，我们不得不在构造这个Counter子类时就遍历一遍宠物。此外，你也可以调用super()方法，传递一个Map进去，而这个Map可以放在静态域中执行完毕，再去构建，不过这样和示例中的做法差异不大。

关键是，现在的解决方案也不好，需要提前将所有元素放入。

下面展示一个使用 `isAssignableFrom` 的通用计数器，这个计数器和Pet完全解耦。

```java
//展示了一个高级的通用的计数器，使用isAssignableFrom判断类型信息
class TypeCounter extends HashMap<Class<?>,Integer> {
    private Class<?> baseType;
    public TypeCounter(Class<?> baseType){ this.baseType = baseType; }
    /**进行判断，如果不属于Class<T>那么抛出错误，否则继续*/
    public void count(Object obj){
        Class<?> type = obj.getClass();
        if (!baseType.isAssignableFrom(type)){
            throw new RuntimeException(obj + "incorrent type:" + type + ". should be type or subtype of" +
                    "" + baseType);
        } countClass(type);
    }
    private void countClass (Class<?> type){
        Integer quantity = get(type);
        put(type,quantity == null ? 1 : quantity + 1 );
        Class<?> superClass = type.getSuperclass();
        if (superClass != null && baseType.isAssignableFrom(superClass))
            countClass(superClass);
    }
    public String toString() {
        StringBuilder sb = new StringBuilder("{");
        for (Map.Entry<Class<?>,Integer> e : entrySet()){
            sb.append(e.getKey().getSimpleName());
            sb.append("="); //不使用 + 的原因是，这是StringBuilder，调用 + 会初始化另一个StringBuilder
            sb.append(e.getValue());
            sb.append(", ");
        } sb.delete(sb.length()-2,sb.length());
        sb.append("}");
        return sb.toString();
    }
}
```

注意到这个计数器 extends了一个泛型的Map，以拜托Pet的限制。之后使用 `isAssignableFrom` 判断一个设定的类对象和我们传递进来对象的Class对象是否有继承，传递过来的是否可以转型为我们定义的？如果是，那么进行技术，这里使用了三元操作符，很方便。这个计数器的用法如下：

```java
TypeCounter tc = new TypeCounter(Pet.class);
for (Pet pet : Pets.createArray(20)){
    tc.count(pet);}
print(); print(tc); //Result: {Rat=1, Cymric=3, Pet=20, Manx=8, Mouse=3...}
```

# 带有RTTI的注册工厂

工厂的作用在之前介绍过。工厂提供了对象，通过工厂接口的调用，可以在不接触对象构造器的前提下得到对象。之前使用匿名类改写过工厂，但是那时的工厂不能处理这个问题：如果带有继承，那么工厂可能陷入无限递归的地步：

下列提供了一个带有协变返回类型的工厂接口。这个工厂可以产生不同种类的对象，而不像之前需要通过转型。之后Part是一个随机产生工厂产品的工厂类，其内部有一个闭包（能够返回工厂产品）的列表。我们对这个闭包实现了工厂接口，然后调用闭包的create()方法返回产品。

```java
interface Factory<T> { T create();}

class Part {
    private static Random rand = new Random();
    static List<Factory<? extends Part>> partfactories =
            new ArrayList<>();
    static {
        partfactories.add(new FueFilter.Callback());
        partfactories.add(new OilFilter.Callback());
        partfactories.add(new FanBelt.Callback());
    }
    public static Part createRandom(){
        int n = rand.nextInt(partfactories.size());
        return partfactories.get(n).create();
    }
    public String toString() {
        return this.getClass().getSimpleName();
    }
}
class Filter extends Part {}

class FueFilter extends Filter {
    public static class Callback implements Factory<FueFilter> {
        public FueFilter create() {
            return new FueFilter();
        }
    }
}

class OilFilter extends Filter {
    public static class Callback implements Factory<OilFilter> {
        public OilFilter create() {
            return new OilFilter();
        }
    }
}

class Belt extends Part {}

class FanBelt extends Belt {
    public static class Callback implements Factory<FanBelt> {
        public FanBelt create() {
            return new FanBelt();
        }
    }
}
```

之所以不对Part直接实现工厂接口的原因，而是采用内部类实现的闭包，是因为，一旦cretae()放在了Part及其继承的内部，那么当我们构造这个工厂列表的时候，就会面临添加自身实例的问题，而这就造成了一个无限的循环。因此，使用闭包代替产生工厂产品的类添加到列表。

而花这么大功夫的原因，就是我们想要在任何继承中都使用 `createRandom` 这个方法产生产品，其中包括自己工厂的产品。因此，当想要使用继承的时候，只能使用闭包来实现接口吗？起码在学习RTTI之前是这样。

下列代码展示了使用类字面量替代闭包来实现继承的工厂模式：

```java
import java.util.*; import static com.mazhangjing.Print.print;

interface Factory<T> { T create();}

class Part implements Factory {
    private static Random rand = new Random();
    static List<Class<? extends Part>> partfactories =
            new ArrayList<>();
    static {
        partfactories.add(FueFilter.class);
        partfactories.add(OilFilter.class);
        partfactories.add(FanBelt.class);
    }
    public static Part createRandom(){
        int n = rand.nextInt(partfactories.size());
        try {
        return partfactories.get(n).getDeclaredConstructor().newInstance();
        } catch (Exception e) { throw new RuntimeException(e);}
    }
    public String toString() {
        return this.getClass().getSimpleName();
    }
    public Object create() {
        return this;
    }
}

class Filter extends Part {}

class FueFilter extends Filter {}

class OilFilter extends Filter {}

class Belt extends Part {}

class FanBelt extends Belt {}
```

显然，这样工作的很好，并且，结构清晰易懂。

根据这个启发，我们对于之前的PetCreator使用了工厂模式，这样就将所有修改集中在一处，当我们需要扩展这个Creator以适配更多的宠物时，只用修改setType方法即可。

```java
//一个很高级的Creator类，展示了使用工厂的PetCreator。类字面常量 + 继承可以实现强大的工厂。
interface PetFactory<T> { T create(); }
class PetF implements PetFactory {
    public String toString() {
        return this.getClass().getSimpleName();
    }
    public Pet create() { return new Pet(); }
}
class PugF extends PetF {
    public Pet create() { return new Pug(); }
}
class RatF extends PetF {
    public Pet create() { return new Rat(); }
}
class FactoryCreator extends PetCreator {
    private static List<Class<? extends PetF>> types = new ArrayList<>();
    private static List<Class<? extends Pet>> typesf = new ArrayList<>();
    private static void setTypes() {
        //对于容器，一定要先初始化，否则不能添加 NoPointerNull!!!!!!!!!!
        types.add(PetF.class);
        types.add(PugF.class);
        types.add(RatF.class);
    }
    static {setTypes();getPugs();}
    private static void getPugs() {
        for (Class<? extends PetF> p : types){
            try {
                typesf.add(p.getDeclaredConstructor().newInstance().create().getClass());
            } catch (Exception e) {
                throw new RuntimeException(e);
            }
        }
    }
    public List<Class<? extends Pet>> types() {
        return typesf;
    }
    public static void main(String[] args){
        print(types);
    }
}
```

# 反射

对于基本的RTTI，Java必须在编译期知道所有的Class类型，这样RTTI才能对其进行识别。但是，有很多时候，我们希望这些类能够在编译后运行的时候再进行加载和初始化以及使用，这对于分布式计算、远程方法调用、快速应用组件开发都有很大的好处。Java使用 Class 类 和 java.lang.reflect 类库支持了反射：运行时获取类信息，加载并且使用。 reflect 类库中主要的类有 `Field Method Constructor`。使用 `get() set()` 获取字段的值并且修改，使用 `invoke()` 执行其方法，使用 `Constructor` 创建新对象等。

这些 reflect 类库中的 Filed Method Constructor 对应的都是空的类，当需要进行加载时，JVM 加载 `.class` 文件，通过反射机制，像普通方法那样使用它们。通过反射的 `.class` 文件均为编译后/运行时打开和检查的，这点和普通的类区别很大。

## 类方法提取器

下面这个例子展示了一个 Java 小程序，这个程序从命令行接受参数作为其需要寻找的类名，然后找到其所有的方法，使用正则表达式过滤后输出到控制台。使用 `javac xxx.java` 编译，使用 `java xxx java.lang.String toString` 来使用这个方法，第一个参数为需要寻找的类，第二个参数为希望进行的过滤。

很显然的，我们使用 forName 寻找这个类对象，使用 `getMethod()` 获取 Method[] 方法列表，使用 `getConstructor()` 获取构造器列表。

```java
import java.lang.reflect.*;
import java.util.regex.*; import static com.mazhangjing.Print.print;

public class ShowMethods {
    private static String usage = "usage:\n" +
            "ShowMethod qualified.class.name\n" +
            "To show all method in class or :\n" +
            "ShowMethod qualified.class.name word\n" +
            "To search for methods involving word";
    private static Pattern p = Pattern.compile("\\w+\\.|native|final");
    public static void main(String[] args){
        if (args.length < 1){
            System.out.print(usage);
            System.exit(0);
        }
        int lines = 0;
        try {
            Class<?> c = Class.forName(args[0]);
            Method[] m = c.getMethods();
            Constructor[] ctors = c.getConstructors();
            if (args.length == 1){
                for (Method i : m){
                    System.out.println(p.matcher(i.toString()).replaceAll(""));
                }
                for (Constructor i : ctors){
                    System.out.println(p.matcher(i.toString()).replaceAll(""));
                }
                lines = m.length + ctors.length;
            } else {
                for (Method i : m){
                    if (i.toString().contains(args[1])){
                        System.out.println(p.matcher(i.toString()).replaceAll(""));
                        lines++;
                    }
                }
                for (Constructor i : ctors){
                    if (i.toString().contains(args[1])){
                        System.out.println(p.matcher(i.toString()).replaceAll(""));
                        lines++;
                    }
                }
            }
        } catch (Exception e){
            System.out.println("No such class: " + e);
        }
    }
}
```

这是一个查找输出：查找String类关于 format 的方法

```cmd
C:\> java ShowMethods java.lang.String format
public static String format(String,Object[])
public static String format(Locale,String,Object[])
```

此外需要注意，当这个类为 public 是，寻找自身可以找到一个并不存在的构造器，而当这个类为 包权限时，则找不到这个构造器，这里的原因是，这构造器和我们的类具有相同的权限等级，如果是包权限，当我们在CMD中寻找时，自然就找不到了，因为其并非 public。

## 动态加载类

下面的实例展示了动态加载一个在运行时才存在的类的方法。

```java
class M {
    Object o;
    M(){}
    M(String c){
        try {
            print("here");
            o = Class.forName(c).getDeclaredConstructor().newInstance(); //动态创建对象的实例
        } catch (Exception e) {throw new RuntimeException(e);}
    }
    public String toString() {
        if (o != null) {return "M with reflect class: " + o.getClass().getSimpleName();}
        else {return "M only!";}
    }
}
class N {
    public String toString() {
        return "N is me!";
    }
}
public static class Ex {
        public static void main(String[] args) {
            M m = new M();
            M m2 = new M("java.lang.String");
            print(m + "\n" + m2);
        }
    }
}
here
M only!
M with reflect class: String
```

# 动态代理

代理是一种常用的方法，但是其缺点在于，对于所有的被代理的类，都需要实现其方法，当这个被代理的类更改时，也要同步更改代理中的内容。而动态代理——利用RTTI，可以很方便解决这个问题。

基于接口的调用是这样工作的：对于一个标准接口的调用（请求方），其通过接口方法调用一个接口的实现的类（服务方）。

而代理则也实现了接口，然后这个调用方传递调用给代理（伪装），代理在其内部实例化一个服务方的对象，然后调用接口实现伪装服务。代理的目的是在中间截获一些动作或者信息。

代理的缺点在于我们必须对服务方的每个方法进行重写，耦合太高。Java通过反射提供了一个叫做InvocationHandler的东西，可以进行动态代理。动态代理包括四个对象，其一请求方，其二代理对象，其三代理处理器，其四，服务方。

- 首先，创建一个服务方对象。

- 代理处理器类必须实现 InvocationHandler 接口，在其 invoke 重载方法中设置要进行的行动：拦截或者监听，然后将数据流发送到服务方。invoke 方法接受三个参数：代理对象、代理的 Method对象，以及参数信息。invoke 应当返回 反射方法对象.invoke(服务方对象，参数)以连接服务方接口进行正常调用。

- 对于代理对象，其必须使用 `Proxy.newProxyInstance()` 创建，其需要接口的类对象的类加载器，一个接口类对象的列表，一个设置好和服务方对象进行交互的代理处理器对象。

`costumer(proxy)-----调用，同时传递参数和方法----->(Interface)proxy-----传递自身以及costumer的参数----->(DynamicProxyHander)xxx----通过在invoked中return方法.invoked(代理，参数)交给服务方处理-------->(ReadObject)ro--------->被调用`

```java
import java.lang.reflect.InvocationHandler;
import java.lang.reflect.Method;
import java.lang.reflect.Proxy;
import static com.mazhangjing.Print.*;

public class ProxyDemo {
    public static void costumer(Interface iface){
        iface.somethingElse("mmmmuum");
        iface.dosomething();
    }
    public static void main(String[] args){
        RealObject ro = new RealObject();
        costumer(ro);//一般基于接口的调用
        //使用Proxy.newProxyInstance()创建一个动态代理。其需要得到一个类加载器（从加载的类的getClassLoader()中获得），
        // 一个希望代理实现的接口列表（必须为接口，不能是类或者抽象类），一个实现了InvocationHandler接口的代理处理类。
        Interface proxy = (Interface) Proxy.newProxyInstance(
                Interface.class.getClassLoader(),
                new Class[]{Interface.class},
                new DynamicProxyHandler(ro)
        );
        //动态代理实例模仿成一个接口，可以被那些希望调用真实实现了接口的类的方法调用
        //经过这种调用，当这个方法costumer调用接口方法时，会自动经过代理对象的Handler的invoke方法。
        costumer(proxy);
    }
}
interface Interface {
    void dosomething();
    void somethingElse(String arg);
}
class RealObject implements Interface {
    public void dosomething() {
        print("dosomething!");
    }
    public void somethingElse(String arg) {
        print("something else" + arg);
    }
}
class DynamicProxyHandler implements InvocationHandler{
    private Object proxied;
    private int count = 0;
    public DynamicProxyHandler(Object proxied){
        this.proxied = proxied;
    }//invoke中传递过来的第一个参数是这个代理对象，第二个参数是希望调用的接口方法，第三个参数是参数
    public Object invoke(Object proxy, Method method, Object[] args) throws Throwable{
        printf("\n**** proxy: \nPROXY: %s\nMETHOD: %s\n"
                    ,proxy.getClass().getSimpleName(),method.getName());
        if (args != null) {
            for (Object arg : args){ printf("ARG: " + arg + "\n"); }
        }
        if (method.getName().equals("somethingElse")){
            printf("\nI'm fucking Proxy, I don't like \"somethingElse\"!, I droped it!\n");
            return null;
        }
        printf("\nPROXY COUNT: %d\n",++count);
                                                                                      
        return method.invoke(proxied, args);
    }
}
```
```
something elsemmmmuum
dosomething!

**** proxy: PROXY: $Proxy0 ; METHOD: somethingElse ; ARG: mmmmuum

I'm fucking Proxy, I don't like "somethingElse"!, I droped it!

**** proxy: PROXY: $Proxy0 ; METHOD: dosomething

PROXY COUNT: 1
dosomething!
```

一般而言，将会默认调用method.invoke请求真实对象处理这个方法。但是，你其实可以像上面那样，对方法等任意进入proxy的东西进行过滤，决定是否要处理。或者就像上面的进行计数，但是这个代理的生命周期如何呢？似乎代理对象对应着服务对象，只要用一个代理对象，那么就不清空计数数据。

此外，动态代理还可以实现事务，当成功时执行提交，当失败时进行回滚。

其实，相比较一般代理，动态代理并没有什么神奇，其依旧需要内置一个实例化的服务方对象。不过，这个对象的方法不需要手动写在代理类中，而是通过Method.invoke进行处理的。此外，我们希望，这个代理类需要实现接口，因此，将这个代理类一分为二，其一为代理类，其二为Handler类，前者用来完成和接口耦合的工作，和调用方交互的工作，而后者用来完成监控和对实例化对象使用Method.invoke的工作。

因为整个代理是基于反射的，因此并没有说，对于Handler同时实现接口，然后让它同时和调用方交互，这个样子工作。而是分成两个部分，前者代理类用来交互，并且用来动态调用接口（估计并没有实现接口，而是在内部动态加载了这个接口的类，然后进行了伪装）。而后者则专门用来进行监控。

# 空对象

空对象可避免对象调用是为null，然后调用其方法会出错的问题。空对象就相当于一个假人模型，其用来执行模拟操作。

```java
import static com.mazhangjing.Print.print;
public class NullObject {
    public static void main(String[] args){
        Person p = null;
        //在这里还看不出空对象的作用，但是，当有一个类包含了一组Person的时候，如果对其进行遍历，那么就不得不添加判断是否为NULL的
        //逻辑，而在Person类中提前定义好NULL模型，在写入列表的时候，如果为NULL，则替换为NULL CLASS INST，这样在之后的处理中
        //就不需要为每一次遍历写判断逻辑了
        if (p == null){
            p = Person.NULL;
        } print(p);
    }
}
class Person {
    private static String first_name;
    private static String last_name;
    private static Integer age;
    Person(String fname,String lname,Integer age){
        this.first_name = fname;
        this.last_name = lname;
        this.age = age;
    }
    public String toString() {
        return String.format("PERSON: %s %s. AGE: %d",first_name,last_name,age);
    }
    public static final Person NULL = new Person("NULL","NULL",0);
}
```

# 接口和类型信息

接口提供了一种抽像而整齐的API，但是RTTI则提供给程序员一种能够随意使用类方法的能力，不论这个类是私有、包可见、内部类、匿名类，得益于RTTI，Java中不存在无法调用的方法，不论权限如何。

总而言之，字节码、包权限、私有方法、内部类、匿名类均不可阻挡任何知道了方法名的非标准调用。而知道方法名，只用javap反汇编即可，唯一的障碍就是猜测方法名是干什么的了。不过，得益于Java一长串的驼峰命名，其实也很好猜。

```java
import java.lang.reflect.Method;
import static com.mazhangjing.Print.*;

//本代码用来展示接口为什么不能阻止类的使用者采用RTTI访问那些私有的方法。
public class InterfaceDemo {
    public static void main(String[] args) throws Exception {
        //展示了即便是使用接口，也无法阻止用户进行向下转型，并且调用那些私有的方法
        M1 m = new M2();
        m.m();//通过基类接口调用
        if (m instanceof M2){
            M2 m2 = (M2)m;
            m2.n();
        }

        //展示了即便声明了包访问权限，我们一样可以使用RTTI跳过限制，访问n()
        m = new M3();
        ((M3) m).n(); //出了包之后，这样写就是错误的。因为包外不存在M3类型可用

        Method n = m.getClass().getDeclaredMethod("n");
        n.setAccessible(true); //m使用RTTI可以找到继承树的任意私有方法，设置可访问，即可调用方法访问。
        n.invoke(m);

        //展示了只保留.class二进制代码，而不提供源文件的限制方法
        //无效，因为 javap --private M3.class 可以直接查看其私有方法名称。

        //展示了使用内部类将实现包裹起来的方法及其无效性
        M4 m4 = new M4();
        M1 m5 = m4.new M5(); //根本没有用到 M4.M5类型，只知道M4和接口，即可突破内部类
        Method n5 = m5.getClass().getDeclaredMethod("n");
        n5.setAccessible(true);
        n5.invoke(m5);

        //展示了使用匿名类是否可以避免使用内部实现的问题
        M6 m6 = new M6();
        Method n6 = m6.m().getClass().getDeclaredMethod("n");
        n6.invoke(m6.m()); //照样被突破了

        //展示了使用private关键字时照样可以被调用
        M1 m2 = new M2();
        Method n_p = m2.getClass().getDeclaredMethod("n_private");
        n_p.setAccessible(true);
        n_p.invoke(m2);
        //当然，除非你声明final，但是，那样几乎是把自己囚禁在笼子里，就没有探讨的必要了。
    }
}

interface M1 {
    void m();
}
//这里应该是 public class ...
class M2 implements M1 {
    public void m() { }
    public void n() { }
    private void n_private() {print("Calling n_private() in M2");}
}
class M3 implements M1 {
    public void m() { }
    public void n() { print("Calling n() in M3");}
}
class M4 {
    class M5 implements M1 {
        public void m() { }
        public void n() { print("Calling n() in M4-M5");}
    }
}
class M6 {
    public M1 m() {
        return new M1() {
            public void m() { }
            public void n() { print("In Minner-n()");}
        };
    }
}
```